## XGBoost

* eXtra Gradient Boost
* 주요 장점
    * 뛰어난 예측 성능
    * GBM 대비 빠른 수행 시간
        * CPU 병렬 처리, GPU 지원
    * 다양한 성능 향상 기능
        * 규제(Regularization) 기능 탑재
        * Tree Pruning(가지치기)
    * 다양한 편의 기능
        * 조기 중단(Early Stopping)
        * 자체 내장된 교차 검증
        * 결손값 자체 처리
* XGBoost 파이썬 구현
    * C/C++ Native Module -> Python Wrapper -> 사이킷런 Wrapper

### XGBoost Python Wrapper와 사이킷런 Wrapper 비교

* Python Wrapper
    1. 사용 모듈
        * from xgboost as xgb
    2. 학습용과 테스트용 데이터 세트
        * DMatrix 객체를 별도로 생성
            * train = xgb.DMartix(X_train, label=y_train)
        * DMartix 생성자로 피처 데이터 세트와 레이블 데이터 세트를 입력
    3. 학습 API
        * Xgb_model = xgb.train()
        * Xgb_model은 학습된 객체를 반환 받음
    4. 예측 API
        * Xgb.train()으로 학습된 객체에서 predict()호출
        * Xgb_model.predict()는 예측 결과가 아니라 예측 결과를 추정하는 확률값 반환
    5. 피처 중요도 시각화
        * plot_importance()함수 이용
* 사이킷런 Wrapper
    1. 사용 모듈
        * from xgboost import XGBClassifier
    2. 학습용과 테스트용 데이터 세트
        * 넘파이나 판다스를 이용
    3. 학습 API
        * XGBClassifier.fit()
    4. 예측 API
        * XGBClassifier.predict()
        * 예측 결과값 반환
    5. 피처 중요도 시각화
        * plot_importance()함수 이용

### XGBoost Python Wrapper와 사이킷런 Wrapper 하이퍼 파라미터 비교

* 사이킷런 Wrapper의 경우 GBM에 동일한 하이퍼 파라미터가 있다면 이를 사용하고 그렇지 않다면, 파이썬 wrapper의 하이퍼 파라미터를 사용

|파이썬 Wrapper|사이킷런 래퍼|하이퍼 파라미터 설명|
|:---|:---|:---|
|eta|learning_rate|GBM의 학습률과 같은 파라미터입니다. 0에서 1사이의 값을 지정하며 부스팅 스탭을 반복적으로 수행할 때 업데이트되는 학습률 값.<br> 파이썬 래퍼 기반의 xgboost를 이용할 경우 디폴트는 0.3. 사이킷런 래퍼 클래스를 이용할 경우 eta는 learning_rate 파라미터로 대체되며, 디폴트는 0.1입니다.|
|num_boost_rounds|n_estimators|사이킷런 앙상블의 n_estimators와 동일. 약한 학습기의 개수(반복 수행 횟수)|
|min_child_weights|min_child_weight|결정트리의 min_child_leaf와 유사. 과적합 조절용|
|max_depth|max_depth|트리의 최대 깊이|
|sub_sample|subsample|GBM의 subsample과 동일. 트리가 커져서 과적합되는 것을 제어하기 위해 데이터를 샘플링하는 비율을 지정합니다.<br> sub_sample=0.5로 지정하면 전체 데이터의 절반을 트리를 생성하는데 사용합니다.<br>0에서 1사이의 값이 가능하나 일반적으로 0.5~1 사이의 값을 사용합니다.|
| lambda | reg_lambda | L2 규제(Regularization) 적용 값. 기본값은 1임. 값이 클수록 규제 값이 커짐. 과적합 제어 |
| alpha | reg_alpha | L1 규제(Regularization) 적용 값. 기본값은 0임. 값이 클수록 규제 값이 커짐. 과적합 제어 |
| colsample_bytree | colsample_bytree | GBM의 max_features와 유사합니다. 트리 생성에 필요한 피처(컬럼)을 임의로 샘플링 하는데 사용됩니다. |
| scale_pos_weight | scale_pos_weight | 특정 값으로 치우친 비대칭한 클래스로 구성된 데이터 세트의 균형을 유지하기 위한 파라미터입니다. 기본값은 1 |
| gamma | gamma | 트리의 리프 노드를 추가적으로 나눌지를 결정할 최소 손실 감소 값입니다. 해당 값보다 큰 손실(loss)이 감소된 경우에 리프 노드를 분리합니다. 값이 클수록 과적합 감소 효과가 있습니다. |

### XGBoost 조기 중단 기능(Early Stopping)

* XGBoost는 특정 반복 횟수만큼 더 이상 비용함수가 감소하지 않으면 지정된 반복횟수를 다 완료하지 않고 수행을 종료할 수 있음
* 학습을 위한 시간을 단축 시킬 수 있음. 특히 최적화 튜닝 단계에서 적절하게 사용 가능
* 너무 반복 횟수를 단축할 경우 예측 성능 최적화가 안된 상태에서 학습이 종료 될 수 있으므로 유의 필요
* 조기 중단 설정을 위한 파라미터
    * early_stopping_rounds : 더 이상 비용 평가 지표가 감소하지 않는 최대 반복횟수
    * eval_metric : 반복 수행 시 사용하는 비용 평가 지표
    * eval_set : 평가를 수행하는 별도의 검증 데이터 세트. 일반적으로 검증 데이터 세트에서 반복적으로 비용 감소 성능 평가